In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.layers import Embedding
from keras import layers
from keras.utils import to_categorical
from keras.regularizers import l2
from keras import Model, Input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import regex as re

In [ ]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
train

In [ ]:
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test

In [ ]:
sub = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sub

In [ ]:
X = [str(train.iloc[i].keyword)+" "+str(train.iloc[i].location)+" "+train.iloc[i].text for i in range(len(train))]
#X = [train.iloc[i].text for i in range(len(train))]
X = [re.sub(r'[^a-zA-Z\ ]', '', s).lower() for s in X]
X = np.array(X)
y = [train.iloc[i].target for i in range(len(train))]
y = np.array(y)
print(np.unique(y, return_counts=True))
#y = to_categorical(y)
X.shape, y.shape, X[0]

In [ ]:
X_test = [str(test.iloc[i].keyword)+" "+str(test.iloc[i].location)+" "+test.iloc[i].text for i in range(len(test))]
#X_test = [test.iloc[i].text for i in range(len(test))]
X_test = [re.sub(r'[^a-zA-Z\ ]', '', s).lower() for s in X_test]
X_test = np.array(X_test)
X_test.shape, X_test[0]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
_, counts = np.unique(y_train, return_counts=True)
class_weight = [np.size(y_train)/c for c in counts]
counts, class_weight

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(128)
vectorizer.adapt(X_train)

In [ ]:
vectorizer.get_vocabulary()[:5]

In [ ]:
output = vectorizer(np.array([["the cat sat on the"]]))
output.numpy()[0, :5]

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(2, len(voc))))

In [ ]:
test = [b"the", b"cat", b"sat", b"on", b"the"]
[word_index[w] for w in test]

In [ ]:
path_to_glove_file = "/kaggle/input/glove6b/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word.decode("utf-8"))
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
X_train = vectorizer(np.array([[s] for s in X_train])).numpy()
X_val = vectorizer(np.array([[s] for s in X_val])).numpy()
X_test = vectorizer(np.array([[s] for s in X_test])).numpy()
X_train.shape, X_val.shape, X_test.shape

In [ ]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
dropout_rate = 0.5
l2_rate = 1.e-6

In [ ]:
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu", kernel_regularizer=l2(l2_rate))(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Dropout(dropout_rate)(x)
x = layers.Conv1D(128, 5, activation="relu", kernel_regularizer=l2(l2_rate))(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Dropout(dropout_rate)(x)
x = layers.Conv1D(128, 5, activation="relu", kernel_regularizer=l2(l2_rate))(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(dropout_rate)(x)
x = layers.Dense(128, activation="relu", kernel_regularizer=l2(l2_rate))(x)
x = layers.Dropout(dropout_rate)(x)
preds = layers.Dense(1, activation="sigmoid", kernel_regularizer=l2(l2_rate))(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

In [ ]:
early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, verbose=1)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])

model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_val, y_val), 
              class_weight={0: class_weight[0], 1: class_weight[1]}, 
              callbacks=[early_stopping, model_checkpoint])

In [ ]:
best_model = load_model("best_model.h5")

In [ ]:
y_pred = best_model.predict(X_test, verbose=1, batch_size=128)

In [ ]:
y_pred = np.round(y_pred)
np.unique(y_pred, return_counts=True)

In [ ]:
for i in range(len(y_pred)):
    sub.iloc[i].target = y_pred[i]
sub

In [ ]:
sub.to_csv("submission.csv", index=False)